In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tensornetworks_pytorch.TNModels import PosMPS, Born
print(torch.__version__)

1.8.0.dev20201128


Use some data

In [5]:
import pickle
with open('datasets/lymphography', 'rb') as f:
        a=pickle.load(f)
X=a[0]
X=X.astype(int)

print("data shape:", X.shape)
print("range of X values:", X.max()+1)

data shape: (148, 19)
range of X values: 8


In [13]:
D = 10
d = 12
mps = PosMPS(D=D, d=d, verbose=True)
rBorn = Born(D=D, d=d, dtype=torch.float, verbose=True) 
cBorn = Born(D=D, d=d, dtype=torch.cfloat, verbose=True)
models = (mps, rBorn, cBorn)
for model in models:
    print(model.core.shape, model.name)
print("===")
for model in models:
    model.fit(X, d)
    
# model = mps
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
# loss_fn = torch.nn.NLLLoss()

# for ep in range(10):
#     print(ep)
#     model.zero_grad()
# #     print(model.core)
#     loss = - model(X[0])
#     loss.backward()
#     optimizer.step()

torch.Size([12, 10, 10]) Positive MPS
torch.Size([12, 10, 10]) Born model torch.float32
torch.Size([12, 10, 10]) Born model torch.complex64
===


In [14]:
batchsize=4
trainloader = DataLoader(X, batch_size=batchsize, shuffle=True)

model = rBorn
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

for batch_idx, batch in enumerate(trainloader):
    model.zero_grad()
    neglogprob = 0
    for x in batch:
        neglogprob -= model(x)
    loss = neglogprob / batchsize
    loss.backward()
    with torch.no_grad():
        print(batch_idx, "\tavg neglogprob:", loss)
    optimizer.step()

0 	avg neglogprob: tensor(inf, grad_fn=<DivBackward0>)
1 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
2 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
3 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
4 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
5 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
6 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
7 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
8 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
9 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
10 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
11 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
12 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
13 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
14 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
15 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
16 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
17 	avg neglogprob: tensor(nan, grad_fn=<DivBackward0>)
18